In [2]:
import httpx
import pandas as pd

base_url = "https://www.gsar.ford.com/onlinereports/be/getVehInfoFull"

In [27]:
url = "https://corp.sts.ford.com/adfs/oauth2/authorize"
params = {
    "client_id": "urn:gsar:clientid:web:prod",
    "response_type": "token+id_token",
    "redirect_uri": "https://www.gsar.ford.com/oauth-callback/&resource=urn:gsar:resource:web:prod&client-request-id=f91c4811-cea1-40e6-c122-0080040c00f8",
}
response = httpx.get(url, params=params)
print(response)

<Response [302 Found]>


In [20]:
def get_token():
    url = "https://corp.sts.ford.com/adfs/oauth2/authorize/wia"
    params = {
        "client_id": "urn:gsar:clientid:web:prod",
        "response_type": "token id_token",
        "redirect_uri": "https://www.gsar.ford.com/oauth-callback/",
        "resource": "urn:gsar:resource:web:prod",
        "client-request-id": "f91c4811-cea1-40e6-c122-0080040c00f8",
        "RedirectToIdentityProvider": "AD AUTHORITY",
    }
    response = httpx.get(url, params=params, follow_redirects=True)
    print(response)

    return "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6ImFSZ2hZU01kbXI2RFZpMTdWVVJtLUJlUENuayJ9.eyJhdWQiOiJ1cm46Z3NhcjpyZXNvdXJjZTp3ZWI6cHJvZCIsImlzcyI6Imh0dHBzOi8vY29ycC5zdHMuZm9yZC5jb20vYWRmcy9zZXJ2aWNlcy90cnVzdCIsImlhdCI6MTcwODY5OTA2NCwiZXhwIjoxNzA4NzI3ODY0LCJDb21tb25OYW1lIjoiVkRVQVJUMTAiLCJzdWIiOiJWRFVBUlQxMCIsInVpZCI6InZkdWFydDEwIiwiZm9yZEJ1c2luZXNzVW5pdENvZGUiOiJGU0FNUiIsImdpdmVuTmFtZSI6IlZpY3RvciIsInNuIjoiRHVhcnRlIiwiaW5pdGlhbHMiOiJWLiIsIm1haWwiOiJ2ZHVhcnQxMEBmb3JkLmNvbSIsImVtcGxveWVlVHlwZSI6Ik0iLCJzdCI6IkJBIiwiYyI6IkJSQSIsImZvcmRDb21wYW55TmFtZSI6IklOU1QgRVVWQUxETyBMT0RJIE4gUkVHSU9OQUwgQkFISUEiLCJmb3JkRGVwdENvZGUiOiIwNjY0Nzg0MDAwIiwiZm9yZERpc3BsYXlOYW1lIjoiRHVhcnRlLCBWaWN0b3IgKFYuKSIsImZvcmREaXZBYmJyIjoiUFJEIiwiZm9yZERpdmlzaW9uIjoiUEQgT3BlcmF0aW9ucyBhbmQgUXVhbGl0eSIsImZvcmRDb21wYW55Q29kZSI6IjAwMDE1ODM4IiwiZm9yZE1hbmFnZXJDZHNpZCI6Im1tYWdyaTEiLCJmb3JkTVJSb2xlIjoiTiIsImZvcmRTaXRlQ29kZSI6IjY1MzYiLCJmb3JkVXNlclR5cGUiOiJFbXBsb3llZSIsImFwcHR5cGUiOiJQdWJsaWMiLCJhcHBpZCI6InVybjpnc2FyOmNsaWVudGlkOndlYjpwcm9kIiwiYXV0aG1ldGhvZCI6Imh0dHA6Ly9zY2hlbWFzLm1pY3Jvc29mdC5jb20vd3MvMjAwOC8wNi9pZGVudGl0eS9hdXRoZW50aWNhdGlvbm1ldGhvZC93aW5kb3dzIiwiYXV0aF90aW1lIjoiMjAyNC0wMi0yM1QxNDo0Mjo0NC44MTFaIiwidmVyIjoiMS4wIn0.dm1oBbsiudh4cT22D4bo719BPpgZXsWyIT5mF_jch47yHJOP5GkSoUgjQtAHOx_5iwU39ugBVxrHCBBS4w6_NDQWrR2nmmmepAYe-6NG6eZWFaJjDTreZ04iKzc46WOiH_uINYozAHyKgYecgtCo2ENc7cDvh-bXtF6GDQlZxCuSnpRxNnEwWwTMCg3YLZZnM5Mqht3eW29ZyY_WYk6fPaz1oOcX089s_3uI3GbDd4HCOUrJ99LdrqgSR4kTr0l4zE2pkT_IWUxp7zJ-quT_r9DJMRsX2qp0QQxb2MWJBU5o-oUm2Sb9xDC31gv7-_3ZC30HMfKnItCTrFm4p7pKRQ"

In [3]:
def get_info_by_vin(vin: str, token: str):
    response = httpx.get(
        base_url,
        params={"vin": vin},
        headers={"Authorization": f"Bearer {token}"},
        proxies={
            "http://": "http://internet.ford.com:83",
            "https://": "http://internet.ford.com:83",
        },
    )
    data = dict(response.json())
    print(data)

    data_needed = ["wersVl", "origWarantDate", "prodDate", "plant", "globVl", "awsVl"]
    filtered = {"vin": vin}

    for key in data_needed:
        if key in data:
            filtered.update({key: data.get(key)})  # type: ignore

    return filtered

In [5]:
vins = [
    "1FTEW1EP2KKE05297",
    "1FTEW1EG4GFC36898",
    "1FM5K8D82KGB03988",
    "1FM5K8HT3GGC10781",
]
token = get_token()

data = [get_info_by_vin(vin, token) for vin in vins]

<Response [401 Unauthorized]>
{'vinCd': '1FTEW1EP2KKE05297', 'mdlYr': '2019', 'fxdYr': '2019', 'vehTypCd': 'T', 'invDealCd': '52904', 'statCd': 800, 'mktDerCd': 'F - FORD', 'wersVl': 'T/FC - P552N F-150 [15-20]', 'awsVl': 'F6 - F150', 'globVl': 'FL - F150 (NA-KCAP)', 'driveCd': 'T/E - 4 WHL L/H PART TIME DRIVE', 'bodyCab': 'T/BC - DOUBLE CAB (CREW CAB)', 'verSer': 'T/AM - 150 SERIES', 'engine': 'T/WQ - 2.7L DOHC 4V DI GT V6 GAS/FFV', 'engGlob': 'E2277 - NANO GAS V6 (GTDI) - LEP', 'engPlant': 'EN22 - LIMA', 'trans': 'T/EU - 10 SPD AUTO TRANSMISSION-10R80', 'transGlob': 'A0884 - AT - 10R80 - LTP', 'transPlant': 'AT08 - A/T LIVONIA', 'plantRegion': 'NA - NORTH AMERICA', 'plantCntry': 'USA - UNITED STATES', 'plant': 'AJ - KANSAS CITY PLANT BUILD', 'prodDate': '10-JUN-2019', 'loadDate': '12-JUN-2019', 'maintDate': '12-DEC-2019', 'salesRegion': 'NA - NORTH AMERICA', 'salesCntry': 'USA - UNITED STATES', 'vehCntFlg': 'Y', 'salesStProv': 'TX', 'salesDeal': 'LEIF JOHNSON FORD TRUCK CITY [152904 

In [ ]:
data = pd.DataFrame(data)

date_obj = pd.to_datetime(data["origWarantDate"], format="%d-%b-%Y")
data["origWarantDate"] = date_obj.dt.strftime("%d/%m/%Y")
date_obj = pd.to_datetime(data["prodDate"], format="%d-%b-%Y")
data["prodDate"] = date_obj.dt.strftime("%d/%m/%Y")

In [36]:
data["origWarantDate"].to_clipboard(excel=True)